In [1]:
import pyvisa

In [2]:
rm = pyvisa.ResourceManager()
rm.list_resources()

('ASRL8::INSTR', 'ASRL13::INSTR')

In [22]:
a = rm.open_resource('ASRL13::INSTR')

In [4]:
def int_to_bytes(i):
    return (i).to_bytes(1, byteorder='big')

In [5]:
def read_registers(arduino):
    print(arduino.query('r'), end='')
    for i in range(23):
        print(arduino.read(), end='')

In [26]:
read_registers(a)

Reading register contents:
   0: 00000000,00000000,00000000,00000000,
   1: 00000000,01000000,00001000,00100000,
   2: 00011111,00111111,11000000,00000000,
   3: 00000000,00000000,01111111,01111111,
   4: 11111111,11111111,11111111,11111111,
   5: 11101001,00100000,01000100,10001001,11000101,11110000,
   6: 10000111,00100100,00000101,10101001,10000000,00101000,
   7: 00000000,00000000,00000000,00000000,
   8: 00000000,00000000,
   9: 00000000,00000000,00000000,00000000,
   A: 00000000,00000000,00000000,00000000,
   B: 10000001,10011001,11000100,00011010,00111001,10100010,00000011,11000100,
   C: 11000101,10100000,10110001,10000101,00000011,11000000,10000011,10110000,
   D: 01001110,11010001,00000000,00011000,
   E: 00001000,10110101,00000000,00000000,00010101,11000010,10001111,01011100,
   F: 00001000,10110101,00000000,00000000,00010101,11000010,10001111,01011100,
  10: 00001000,10110101,00000000,00000000,00010101,11000010,10001111,01011100,
  11: 00001000,10110101,00000000,00000000,00

In [18]:
def write_register(arduino, reg, data_list):
    # send the write command
    arduino.write_raw(
        b"w" \
          + bytes(bytearray([reg])) \
          + bytes(bytearray(data_list))
      )
    
    # print Arduino's return text
    print(arduino.read(), end='')
    print(arduino.read(), end='')

In [25]:
write_register(a, 0x02, [0b00011111, 0b00111111, 0b11000000, 0b00000000])
write_register(a, 0x0E, [0b00001000, 0b10110101, 0b00000000, 0b00000000, 0b00010101, 0b11000010, 0b10001111, 0b01011100])

Writing to register 2:   
  00011111,00111111,11000000,00000000,
Writing to register E:   
  00001000,10110101,00000000,00000000,00010101,11000010,10001111,01011100,


In [21]:
a.close()